In [1]:
import json
import pandas as pd
import numpy as np
import requests
import time
import datetime as dt

In [2]:
#do I want to update from a preexisting csv?
update = False
#Which subreddit?
subreddit = 'Bitcoin'

In [3]:
def epoch_to_date(epoch_time):
    return dt.datetime.fromtimestamp(epoch_time).strftime('%Y-%m-%d %H:%M:%S')

def date_to_epoch(df_date):
    pattern = '%Y-%m-%d %H:%M:%S'
    return int(time.mktime(time.strptime(df_date, pattern)))

def rq(moving_date):
    #generates a request before 'moving_date'
    #if update. requests before 'moving' and after 'last_date'
    if update==False:
        r = requests.get("https://api.pushshift.io/reddit/search/submission/"+
             "?subreddit={}&size=500&is_video=False&before={}".format(subreddit,moving_date))
    else:
        r =  requests.get("https://api.pushshift.io/reddit/search/submission/"+
             "?subreddit={}&size=500&is_video=False&before={}&after={}".format(subreddit,moving_date,last_date))
    return r

In [4]:
if update==True:
    #imports previous csv
    df_0 = pd.read_csv('./Data/Reddit_{}_PushShift.csv'.format(subreddit),index_col=0)    
    #finds 24h before most recent value
    last_date = date_to_epoch(df_0.date.max())-86400 
    #reassigns df to df before 24h before last update
    #as to assume there may have been changes to 24h posts
    df_0 = df_0[df_0['date'] > epoch_to_date(last_date)]

In [ ]:
#Starts as 'now', but moves with iteration
moving_date = round(time.time())
init = True
n_ls = []
df = pd.DataFrame()

while (init or len(r.json()['data'])>0):
    init = False
    print('Searching Dates before {}'.format(epoch_to_date(moving_date)))
    #print(len(r.json()['data']))
    print()
    
    r = rq(moving_date)
    #pushshifts limit is 200r/min
    # @ 1.5 sec, we are under 100r/min
    time.sleep(2)

    for sub in r.json()['data']:
        n_dic = {}
        n_dic['date'] = epoch_to_date(sub['created_utc'])
        n_dic['author'] = sub['author']
        n_dic['comments'] = sub['num_comments']
        n_dic['score'] = sub['score']
        n_dic['title'] = sub['title']
        n_dic['url'] = sub['url']
        n_ls.append(n_dic)
    df = df.append(pd.DataFrame(n_ls))

    moving_date = date_to_epoch(df.date.min())

Searching Dates before 2018-08-26 09:57:58

Searching Dates before 2018-08-24 09:28:24

Searching Dates before 2018-08-22 19:15:40

Searching Dates before 2018-08-21 11:40:20

Searching Dates before 2018-08-19 23:45:10

Searching Dates before 2018-08-17 22:05:39

Searching Dates before 2018-08-16 09:01:50

Searching Dates before 2018-08-14 22:30:51

Searching Dates before 2018-08-13 18:25:03

Searching Dates before 2018-08-12 05:59:10

Searching Dates before 2018-08-10 12:17:22

Searching Dates before 2018-08-09 06:54:31

Searching Dates before 2018-08-08 00:53:18

Searching Dates before 2018-08-06 13:51:45

Searching Dates before 2018-08-04 15:56:10

Searching Dates before 2018-08-03 05:21:26

Searching Dates before 2018-08-01 16:55:06

Searching Dates before 2018-07-31 08:26:15

Searching Dates before 2018-07-29 23:52:29

Searching Dates before 2018-07-28 00:22:33

Searching Dates before 2018-07-26 14:45:36

Searching Dates before 2018-07-25 07:45:36

Searching Dates before 2018-07-2

In [ ]:
if update:
    df = df.append(df_0).sort_values('date',ascending=True)
df.to_csv('./Data/Reddit_{}_PushShift.csv'.format(subreddit))